In [56]:
import cmath
import math
from IPython.display import HTML,display


def calculate_pvalue(Vmatrix,Ybus,n):
    VofP = Vmatrix[n]
    polarVofP= cmath.polar(VofP)
    modVofP = polarVofP[0]
    phaseVofP = polarVofP[1]
    
    P=0
    
    for (index,v) in enumerate(Vmatrix):
        polarVother = cmath.polar(v)
        modVother = polarVother[0]
        phaseVother = polarVother[1]
        
        
        polarY = cmath.polar(Ybus[n][index])
        modpolarY = polarY[0]
        phasepolarY = polarY[1]
        a=modVofP*modVother*modpolarY*math.cos(phasepolarY+phaseVother-phaseVofP)
        P+=a
    print("P is " + str(P))
    return P


def calculate_qvalue(Vmatrix,Ybus,n):
    VofP = Vmatrix[n]
    polarVofP= cmath.polar(VofP)
    modVofP = polarVofP[0]
    phaseVofP = polarVofP[1]
    
    Q=0
    
    for (index,v) in enumerate(Vmatrix):
        polarVother = cmath.polar(v)
        modVother = polarVother[0]
        phaseVother = polarVother[1]
        
        
        polarY = cmath.polar(Ybus[n][index])
        modpolarY = polarY[0]
        phasepolarY = polarY[1]
        a=-modVofP*modVother*modpolarY*math.sin(phasepolarY+phaseVother-phaseVofP)
        Q+=a
    print("Q is " + str(Q))
    return Q











    

In [67]:
def find_p_and_q_matrices(p_list,q_list,Vmatrix,Ybus):#plist and q list are lists of indices
    cq = []
    for m in p_list:
        cq.append(calculate_pvalue(Vmat,bus0,m-1))
    for m in q_list:
        cq.append(calculate_qvalue(Vmat,bus0,m-1))
    print("Calculated quantities are" + str(cq))
    return cq

        
    
def get_mismatch_p_matrix(matA,matB):
    if len(matA)!=len(matB):
        print("Error lengths don't match")
        return
    matC=[]
    for (index,value) in enumerate(matA):
        matC.append(matA[index]-matB[index])
    print("MAtC is"+str(matC))
    return matC


def get_delta_p_by_mod_v_matrix(deltaPmatrix,indices,Vmatrix):
    if len(deltaPmatrix)!=len(indices):
        print("Error lengths don't match")
        return
    matC =[]
    
    for (index,m) in enumerate(indices):
        a =deltaPmatrix[index]/abs(Vmatrix[m-1])
        matC.append(a)
    return matC
    
    
    

In [62]:
import numpy as np
def get_fast_decouple_step(Bmatrix,dp_by_modV):
    np_Bmatrix = np.array(Bmatrix)
    np_dp_modV = np.array(dp_by_modV)
    out = np.matmul(np.linalg.inv(np_Bmatrix),np_dp_modV)
    print(out)
    return out
    

In [26]:
def find_pmatrices(Ybus,Pvalues,Qvalues):
    html = "<font face = \"consolas\">"
    html+="<table>"
    for m in Ybus:
        html +="<tr>"
        for n in m:
            val = cmath.polar(n)
            magnitude = val[0]
            phase_in_deg = val[1]*180/cmath.pi
            html+= "<td>%s<td>"%(str(magnitude)+"∠"+str(phase_in_deg))
        html += "</tr>"
    html+= "</table>"
    display(HTML(html))

In [27]:
bus = [[-7.5j ,4j,2.5j,0],[4j,-7.75j,0,2.5j],[2.5j,0,-4.5j,2j],[0,2.5j,2j,-4.5j]]
find_pmatrices(bus,0,0)

In [30]:
V0=1+0j
Vmat = [1.02+0j,1+0j,1.03+0j]
bus0 = [[15-55j,-5+15j,-10+40j],[-5+15j,20-65j,-15+50j],[-10+40j,-15+50j,25-90j]]
find_pmatrices(bus0,0,0)

In [57]:
calculate_pvalue(Vmat,bus0,1) #(Matrix of assumed Vs,Bus matrix ,and index of P to calc)
calculate_qvalue(Vmat,bus0,1)
find_p_and_q_matrices([2,3],[2],Vmat,bus0)

P is -0.5500000000000007
Q is -1.79999999999999
P is -0.5500000000000007
P is 0.5664999999999978
Q is -1.79999999999999
Calculated quantities are[-0.5500000000000007, 0.5664999999999978, -1.79999999999999]


[-0.5500000000000007, 0.5664999999999978, -1.79999999999999]

In [65]:
matA = [-2,1.5]
matB = find_p_and_q_matrices([2,3],[],Vmat,bus0)
get_mismatch_p_matrix(matA,matB)

P is -0.5500000000000007
P is 0.5664999999999978
Calculated quantities are[-0.5500000000000007, 0.5664999999999978]
MAtC is[-1.4499999999999993, 0.9335000000000022]


[-1.4499999999999993, 0.9335000000000022]

In [71]:
c =  get_delta_p_by_mod_v_matrix(get_mismatch_p_matrix(matA,matB),[2,3],Vmat)
B_matrix = [[-65,50],[50,-90]]
get_fast_decouple_step(B_matrix,c)


MAtC is[-1.4499999999999993, 0.9335000000000022]
[0.0254282  0.00405666]


array([0.0254282 , 0.00405666])

In [ ]:
#todo design fast decoupled method 